In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
import os
import sys 

project_path = os.path.join(os.getcwd(), "..", "..")
sys.path.append(project_path)

from utils.transformations import reusable

### **DimUser processing**


### **AUTOLOADER** 

In [0]:
df_user = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format", "parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimUser/checkpoint")\
    .option("schemaEvolutionMode", "addNewColumns")\
    .load("abfss://bronze@deazurestorageproject.dfs.core.windows.net/DimUser")

In [0]:
display(df_user)

In [0]:
df_user = df_user.withColumn("user_name", upper(col("user_name")))
display(df_user)

In [0]:
df_user_obj = reusable()

df_user = df_user_obj.dropCols(df_user, ['_rescued_data'])

df_user = df_user.dropDuplicates(["user_id"])

display(df_user)

In [0]:
df_user.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimUser/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimUser/Data")\
    .toTable("spotify_catalog.silver.DimUser")

### **DimArtist processing**


In [0]:
df_artist = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimArtist/checkpoint")\
    .option("schemaEvolutionMode","addNewColumns")\
    .load("abfss://bronze@deazurestorageproject.dfs.core.windows.net/DimArtist")

display(df_artist)

In [0]:
df_artist_obj = reusable()

df_artist = df_artist_obj.dropCols(df_artist, ['_rescued_data'])

df_artist = df_artist.dropDuplicates(["artist_id"])

display(df_artist)

In [0]:
df_artist.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimArtist/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimArtist/Data")\
    .toTable("spotify_catalog.silver.DimArtist")

### **DimTrack processing**


In [0]:
df_track = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@deazurestorageproject.dfs.core.windows.net/DimTrack/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
    .load("abfss://bronze@deazurestorageproject.dfs.core.windows.net/DimTrack")


In [0]:
df_track_obj = reusable()

df_track = df_track_obj.dropCols(df_track, ['_rescued_data'])

df_track = df_track.withColumn("duration_flag", when(col("duration_sec") < 150, "short")\
    .when(col("duration_sec") < 300, "medium")\
    .otherwise("long"))

df_track = df_track.withColumn("track_name", regexp_replace(col("track_name"), '-', ' '))

display(df_track)

In [0]:
df_track.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimTrack/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimTrack/Data")\
    .toTable("spotify_catalog.silver.DimTrack")

### **DimDate processing**



In [0]:
df_date = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@deazurestorageproject.dfs.core.windows.net/DimDate/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
    .load("abfss://bronze@deazurestorageproject.dfs.core.windows.net/DimDate")

display(df_date)

In [0]:
df_date = reusable().dropCols(df_date, ["_rescued_data"])

In [0]:
df_date.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimDate/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@deazurestorageproject.dfs.core.windows.net/DimDate/Data")\
    .toTable("spotify_catalog.silver.DimDate")

### ***FactStream processing***

In [0]:
df_fact = spark.readStream.format("cloudFiles")\
    .option("cloudFiles.format","parquet")\
    .option("cloudFiles.schemaLocation","abfss://silver@deazurestorageproject.dfs.core.windows.net/FactStream/checkpoint")\
    .option("cloudFiles.schemaEvolutionMode","addNewColumns")\
    .load("abfss://bronze@deazurestorageproject.dfs.core.windows.net/FactStream")

In [0]:
display(df_fact)

In [0]:
df_fact = reusable().dropCols(df_fact, ["_rescued_data"])


In [0]:
df_fact.writeStream.format("delta")\
    .outputMode("append")\
    .option("checkpointLocation", "abfss://silver@deazurestorageproject.dfs.core.windows.net/FactStream/checkpoint")\
    .trigger(once=True)\
    .option("path", "abfss://silver@deazurestorageproject.dfs.core.windows.net/FactStream/Data")\
    .toTable("spotify_catalog.silver.FactStream")